In [106]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score, classification_report

from sklearn.model_selection import train_test_split,  KFold, cross_val_score
from sklearn.neural_network import MLPRegressor, MLPClassifier

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
train = pd.read_csv('../data/data3_train.csv')
test = pd.read_csv('../data/data3_test.csv')

In [3]:
target_column = "shot_made_flag"  # y_column_name

In [70]:
def confusion(true, pred):
    """
    Function for pretty printing confusion matrices
    """
    pred = pd.Series(pred)
    true = pd.Series(true)
    
    true.name = 'target'
    pred.name = 'predicted'
    cm = pd.crosstab(true.reset_index(drop=True), pred.reset_index(drop=True))
    print(cm)
    cm = cm[cm.index]
    return cm

In [4]:
def compute_metrics(y_true,y_pred):
    accuracy = accuracy_score(y_true,y_pred)
    f1_score_1 = f1_score(y_true,y_pred,average='binary',pos_label=1)
    f1_score_0 = f1_score(y_true,y_pred,average='binary',pos_label=0)
    f1_score_macro = f1_score(y_true,y_pred,average='macro')
    return [accuracy,f1_score_1,f1_score_0,f1_score_macro]
    
    

results = pd.DataFrame(columns=['Accuracy', 'F1-score (class 1)', 'F1-score (class 0)', 'F1-score (macro avg)'])

In [5]:
def split_x_y(df, target):
    y = df.loc[:, [target]]
    x = df.drop([target], axis=1)
    return x, y

def split_regular_playoff(df, season, train=None, test=None, validation=None):

    # if(train + test + validation != 1):
    #     print("Error dataset split")
    #     return None

    df_validaiton = None

    df_season = df[df['season'] == season]
    isPlayoffs = df_season.playoffs == 1

    df_train = df_season[~isPlayoffs]
    df_test  = df_season[isPlayoffs]

    print(f"Size of training dataset {len(df_train)}" )
    print(f"Size of test dataset {len(df_test)}" )
    print(f"Size of train vs test ratio {len(df_train) / (len(df_train) + len(df_test)) }" )

    return df_train, df_test
#     train = split_x_y(df_train, 'shot_made_flag')
#     test = split_x_y(df_test, 'shot_made_flag')
#     return train[0], train[1], test[0], test[1]
#     return train, test

# x_train, y_train, x_test, y_test = split_regular_playoff(df, '2010-11')
# train, test = split_regular_playoff(df, '2010-11')

In [6]:
train.head()

,loc_x,loc_y,period,playoffs,shot_distance,shot_made_flag,shot_zone_basic,shot_zone_range,time_remaining,last_5_games_avg,...,shot_type_2PT Field Goal,shot_type_3PT Field Goal,shot_zone_area_Back Court(BC),shot_zone_area_Center(C),shot_zone_area_Left Side Center(LC),shot_zone_area_Left Side(L),shot_zone_area_Right Side Center(RC),shot_zone_area_Right Side(R),matchup_away,matchup_home
0,0,0,3,0,0,0.0,0,0,225,0.0,...,1,0,0,1,0,0,0,0,0,1
1,0,0,3,0,0,0.0,0,0,429,0.0,...,1,0,0,1,0,0,0,0,0,1
2,144,41,3,0,14,1.0,2,1,159,0.0,...,1,0,0,0,0,0,0,1,0,1
3,-161,132,3,0,20,0.0,2,2,49,0.0,...,1,0,0,0,1,0,0,0,0,1
4,34,82,4,0,8,0.0,1,1,596,0.0,...,1,0,0,1,0,0,0,0,0,1


## Split X & Y

In [7]:
x_train, y_train = split_x_y(train, target_column)
x_test, y_test = split_x_y(test, target_column)

In [8]:
lambdas = [1e-10,1e-5,1e-4,1e-3,1e-2,0.1, 0.5,1,5,10,50,100]

# Linear Regression

In [11]:
reg = LinearRegression()
a = reg.fit(x_train, y_train)
reg.score(x_train, y_train)

0.2299316121604148

#### train data

In [12]:
y_train_pred = reg.predict(x_train)
y_train_pred = y_train_pred >= 0.6
y_train_pred = np.multiply(y_train_pred, 1)
accuracy_score(y_train.values, y_train_pred)

0.7150297619047619

#### Test Data

In [14]:
y_test_pred = reg.predict(x_test)
y_test_pred = y_test_pred >= 0.5
y_test_pred = np.multiply(y_test_pred, 1)
accuracy_score(y_test.values, y_test_pred)
results.loc['Linear',:] = compute_metrics(y_test.values, y_test_pred)
results

0.6799065420560748

,Accuracy,F1-score (class 1),F1-score (class 0),F1-score (macro avg)
Linear,0.679907,0.559486,0.748624,0.654055


# Lasso Regression

In [15]:
lasso_cv = LassoCV(alphas=lambdas, cv=5, max_iter=100_000)
a = lasso_cv.fit(x_train.values, y_train.values.flatten())

lasso_r2 =  np.mean(cross_val_score(lasso_cv, x_train.values, y_train.values.flatten()))

print('Best lambda:', lasso_cv.alpha_, 'R2 score:',lasso_r2)

/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.23965911904753057, tolerance: 0.026618604651162787
  model = cd_fast.enet_coordinate_descent_gram(
/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 33.88716132641106, tolerance: 0.026703345724907065
  model = cd_fast.enet_coordinate_descent_gram(


LassoCV(alphas=[1e-10, 1e-05, 0.0001, 0.001, 0.01, 0.1, 0.5, 1, 5, 10, 50, 100],
        cv=5, max_iter=100000)

/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.7471318485644645, tolerance: 0.02138093023255813
  model = cd_fast.enet_coordinate_descent_gram(
/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 27.540648105934764, tolerance: 0.021294883720930215
  model = cd_fast.enet_coordinate_descent_gram(
/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.2317315744502082, tolerance: 0.02134081395348837
  model = cd_fast.enet_coordinate_descent_gram(
/home/teemo/anaconda3/envs/ds/lib/pytho

Best lambda: 0.001 R2 score: 0.1925524666578287


/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26.167473087145524, tolerance: 0.02131521486643436
  model = cd_fast.enet_coordinate_descent_gram(


In [16]:
y_train_pred = lasso_cv.predict(x_train.values)
y_train_pred = y_train_pred >= 0.5
y_train_pred = np.multiply(y_train_pred, 1)
accuracy_score(y_train.values, y_train_pred)

0.7120535714285714

In [17]:
y_test_pred = lasso_cv.predict(x_test.values)
y_test_pred = y_test_pred >= 0.5
y_test_pred = np.multiply(y_test_pred, 1)
accuracy_score(y_test.values, y_test_pred)
results.loc['Lasso',:] = compute_metrics(y_test.values, y_test_pred)
results

0.6869158878504673

,Accuracy,F1-score (class 1),F1-score (class 0),F1-score (macro avg)
Linear,0.679907,0.559486,0.748624,0.654055
Lasso,0.686916,0.570513,0.753676,0.662095


# Ridge Regression

In [18]:
ridge_cross_val_metrics = pd.DataFrame(columns=['mean MSE', 'mean norm_MSE', 'mean R2'])
# We calculate the cross-validation metrics for each lambda 
for lambda_val in lambdas:
    kf = KFold(n_splits=5)
    i=1
    cv_mse = []
    cv_nmse = []
    cv_r2 = []
    # We compute the metrics for each fold and then perform the mean.
    for train_index, test_index in kf.split(x_train):
        x_train_fold = x_train.values[train_index]
        y_train_fold = y_train.values[train_index]
        x_test_fold = x_train.values[test_index,:]
        y_test_fold = y_train.values[test_index]

        ridge = Ridge(alpha=lambda_val)
        a = ridge.fit(x_train_fold,y_train_fold)
        y_pred_fold = ridge.predict(x_test_fold)
        fold_mse = mean_squared_error(y_test_fold, y_pred_fold)
        fold_nmse =  1-r2_score(y_test_fold, y_pred_fold)
        fold_r2 = r2_score(y_test_fold, y_pred_fold)
        cv_mse.append(fold_mse)
        cv_nmse.append(fold_nmse)
        cv_r2.append(fold_r2)
    ridge_cross_val_metrics.loc['Lambda={}'.format(lambda_val),:] = [np.mean(cv_mse),np.mean(cv_nmse),np.mean(cv_r2)]

ridge_cross_val_metrics.loc['Mean', :] = ridge_cross_val_metrics.mean()
ridge_cv_r2 = ridge_cross_val_metrics.loc["Mean", "mean R2"]

ridge_cross_val_metrics.sort_values(by='mean R2',ascending=False)

/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.00747e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.07506e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.01917e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.03564e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/teemo/anaconda3/en

,mean MSE,mean norm_MSE,mean R2
Lambda=5,0.20011,0.810713,0.189287
Lambda=10,0.200881,0.813862,0.186138
Lambda=1,0.201261,0.815277,0.184723
Lambda=0.5,0.201932,0.817959,0.182041
Lambda=0.1,0.202772,0.821318,0.178682
Lambda=0.01,0.203019,0.822308,0.177692
Lambda=0.001,0.203045,0.822413,0.177587
Lambda=0.0001,0.203048,0.822424,0.177576
Lambda=1e-05,0.203048,0.822425,0.177575
Lambda=1e-10,0.203049,0.822426,0.177574


In [19]:
temp_df = ridge_cross_val_metrics.sort_values(by='mean R2',ascending=False)
temp_df.iloc[0]

mean MSE          0.20011
mean norm_MSE    0.810713
mean R2          0.189287
Name: Lambda=5, dtype: object

In [20]:
ridge = Ridge(alpha=5)

ridge.fit(x_train.values, y_train.values.flatten())

y_train_pred = ridge.predict(x_train)
y_train_pred = y_train_pred >= 0.5
y_train_pred = np.multiply(y_train_pred, 1)
accuracy_score(y_train.values, y_train_pred)

Ridge(alpha=5)

/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(


0.7142857142857143

In [21]:
y_test_pred = ridge.predict(x_test)
y_test_pred = y_test_pred >= 0.5
y_test_pred = np.multiply(y_test_pred, 1)
accuracy_score(y_test.values, y_test_pred)
results.loc['Ridge',:] = compute_metrics(y_test.values, y_test_pred)
results

/home/teemo/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(


0.6845794392523364

,Accuracy,F1-score (class 1),F1-score (class 0),F1-score (macro avg)
Linear,0.679907,0.559486,0.748624,0.654055
Lasso,0.686916,0.570513,0.753676,0.662095
Ridge,0.684579,0.560261,0.754098,0.657179


# Neural Network

In [123]:
regr = MLPClassifier(hidden_layer_sizes=(12, ), learning_rate_init=0.00001,  max_iter=1_000_000, tol=1e-8)
regr.fit(x_train.values, y_train.values.flatten())
y_train_pred = regr.predict(x_train.values)
regr.score(x_train.values, y_train.values.flatten())

MLPClassifier(hidden_layer_sizes=(12,), learning_rate_init=1e-05,
              max_iter=1000000, tol=1e-08)

0.6644345238095238

In [124]:
y_train_pred = regr.predict(x_train.values)
y_train_pred = y_train_pred >= 0.5
y_train_pred = np.multiply(y_train_pred, 1)
accuracy_score(y_train.values, y_train_pred)

0.6644345238095238

In [125]:
y_test_pred = regr.predict(x_test.values)
y_test_pred = y_test_pred >= 0.5
y_test_pred = np.multiply(y_test_pred, 1)
accuracy_score(y_test.values, y_test_pred)
results.loc['Neural Net',:] = compute_metrics(y_test.values, y_test_pred)
results

0.6285046728971962

,Accuracy,F1-score (class 1),F1-score (class 0),F1-score (macro avg)
Linear,0.679907,0.559486,0.748624,0.654055
Lasso,0.686916,0.570513,0.753676,0.662095
Ridge,0.546729,0.04902,0.702454,0.375737
Neural Net,0.628505,0.519637,0.697143,0.60839


In [126]:
# confusion(y_test.values.flatten(), y_test_pred)
confusion(y_test_pred, y_test.values.flatten())

print( classification_report(y_test.values, y_test_pred))

predicted  0.0  1.0
target             
0          183  108
1           51   86


predicted,0.0,1.0
target,,
0,183,108
1,51,86


              precision    recall  f1-score   support

         0.0       0.63      0.78      0.70       234
         1.0       0.63      0.44      0.52       194

    accuracy                           0.63       428
   macro avg       0.63      0.61      0.61       428
weighted avg       0.63      0.63      0.62       428

